In [1]:
# !git clone https://github.com/Adrian-Cespedes/Proy4-ML.git

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, TensorDataset

from matplotlib import rcParams
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

np.random.seed(2024)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ruta = '/content/drive/MyDrive/DataProyect/'
df_train = pd.read_pickle(ruta +'df_train_processed.pkl')
df_submit = pd.read_pickle(ruta +'df_test_processed.pkl')

In [5]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_train, test_size=0.2, random_state=2024)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(max_features=1000)
tfidf = TfidfVectorizer()

x_train = tfidf.fit_transform(df_train['lemmas'].apply(' '.join))
x_test = tfidf.transform(df_test['lemmas'].apply(' '.join))
x_submit = tfidf.transform(df_submit['lemmas'].apply(' '.join))

In [7]:
x_train.shape

(20000, 62496)

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['label'])
y_test = label_encoder.transform(df_test['label'])


#verificar que significa cada valor numerico
label_encoder.classes_

array(['neg', 'pos'], dtype=object)

In [9]:
from sklearn.feature_selection import SelectKBest, chi2

k = 5000  # Número de características a seleccionar, ajusta este valor según sea necesario
chi2_selector = SelectKBest(chi2, k=k)
x_train_chi2 = chi2_selector.fit_transform(x_train, y_train)
x_test_chi2 = chi2_selector.transform(x_test)
x_submit_chi2 = chi2_selector.transform(x_submit)

In [10]:
n_features = x_train_chi2.shape[1]
print(f'Number of features: {n_features}')

Number of features: 5000


In [11]:
x_train_chi2.shape

(20000, 5000)

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Definir la clase del modelo RNN
class SentimentRNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, output_dim, dropout):
        super(SentimentRNN, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(hidden[-1, :, :])
        return self.fc(hidden)

In [14]:
# Definir los parámetros del modelo
INPUT_DIM = x_train_chi2.shape[1]
EMBEDDING_DIM = 100
HIDDEN_DIM = 64
NUM_LAYERS = 1
OUTPUT_DIM = len(label_encoder.classes_)
DROPOUT = 0.5


# Inicializar el modelo
model = SentimentRNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, NUM_LAYERS, OUTPUT_DIM, DROPOUT).to(device)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [15]:

# Definir la función de pérdida y el optimizador
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005)



In [20]:
def train(model, loss_f, optimizer, epochs, train_loader):
    loss_lista = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            y_pred = model(x_batch)
            loss = loss_f(y_pred, y_batch)
            epoch_loss += loss.item()

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print(f'Epoch [{epoch+1}/{epochs}], Batch Loss: {loss.item():.4f}')

        avg_epoch_loss = epoch_loss / len(train_loader)
        loss_lista.append(avg_epoch_loss)
        print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {avg_epoch_loss:.4f}')

    return loss_lista

In [17]:
from sklearn.metrics import confusion_matrix, classification_report
def matriz_confusion(y_test, y_pred, tipo):
    matrix = confusion_matrix(y_test, y_pred)
    f2 = pd.DataFrame(matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis], index=label_encoder.classes_, columns=label_encoder.classes_)
    sns.heatmap(f2, annot=True, cbar=None, cmap="Greens")
    plt.title("Confusion Matrix - " + tipo)
    plt.tight_layout()
    plt.xlabel("Predicted")
    plt.ylabel("Real")
    plt.show()

def test(model, x_test, y_test):
    model.eval()
    with torch.no_grad():
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        y_pred = model(x_test)
        y_pred = torch.argmax(y_pred, dim=1).cpu()
        y_test = y_test.cpu()

    print(classification_report(y_test, y_pred))
    matriz_confusion(y_test, y_pred, "RNN")

In [18]:
x_train_dv = torch.tensor(x_train_chi2.toarray(), dtype=torch.long).to(device)
y_train_dv = torch.tensor(y_train, dtype=torch.long).to(device)
train_dataset = TensorDataset(x_train_dv, y_train_dv)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [21]:
print("RNN TRAIN")
loss_list = train(model, loss, optimizer, 10, train_loader)

RNN TRAIN


Epoch 1/10:   1%|          | 1/157 [00:18<47:05, 18.11s/it]

Epoch [1/10], Batch Loss: 0.6988


Epoch 1/10:   1%|▏         | 2/157 [00:32<41:48, 16.18s/it]

Epoch [1/10], Batch Loss: 0.6859


...

Epoch [2/10], Batch Loss: 0.6925


Epoch 2/10:  56%|█████▌    | 88/157 [26:58<21:27, 18.66s/it]

Epoch [2/10], Batch Loss: 0.6926


Epoch 2/10:  56%|█████▌    | 88/157 [27:16<21:23, 18.60s/it]


KeyboardInterrupt: 

Cloning into 'Proy4-ML'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 19), reused 29 (delta 8), pack-reused 0
Receiving objects: 100% (43/43), 34.08 MiB | 16.82 MiB/s, done.
Resolving deltas: 100% (19/19), done.


/bin/bash: line 1: fg: no job control
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
